# 1. 베이지안 최적화 기법 (25점)

1-1 XGBoost , LightGBM 하이퍼 하라미터 튜닝 시에 GridSearch 방식보다는 베이지안 최적화 기법적용하고는 하는데 그 이유는 무엇일까요? (5점)

XGBoost와 LGBM은 하이퍼 파라미터 개수가 많은 특징을 가지는데 GridSearch방식은 하이퍼 파라미터의 개수가 많을 경우 최적화 수행 시간이 오래 걸리기 때문에

1-2 아래 코드를 실행하고 문제를 풀어주세요 (20점)

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

dataset = load_wine()

wine = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
wine['target']= dataset.target
X_features = wine.iloc[:, :-1]
y_label =  wine.iloc[:, -1]

# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label, test_size=0.2, random_state=156 )

# 앞에서 추출한 학습 데이터를 다시 학습과 검증 데이터로 분리
X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train, test_size=0.1, random_state=156 )

(1) hyperopt 을 이용하여 max_depth는 5에서 15까지 1간격으로, min_child_weight는 1에서 2까지 1간격으로 colsample_bytree는 0.5에서 0.8사이, learning_rate는 0.01에서 0.2사이 정규 분포된 값으로 검색하도록 지정해주세요 (10점)

In [2]:
# !pip install hyperopt

In [3]:
from hyperopt import hp

# max_depth는 5에서 20까지 1간격으로, min_child_weight는 1에서 2까지 1간격으로
# colsample_bytree는 0.5에서 1사이, learning_rate는 0.01에서 0.2 사이 정규 분포된 값으로 검색.
xgb_search_space = {'max_depth' : hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight' : hp.quniform('min_child_weight', 1, 2, 1),
                    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 0.8),
                    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2)}

(2) 아래의 코드를 실행한후 objective_func() 함수와 fmin() 함수를 이용하여 최대 반복 수행횟수는 40회로, rstate는 np.random.default_rng(seed=9)) 로 설정하여 최적의 하이퍼 파라미터를 출력해주세요 (10점)

In [4]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

# fmin()에서 입력된 search_space 값으로 입력된 모든 값은 실수형임.
# XGBClassifier의 정수형 하이퍼 파라미터는 정수형 변환을 해줘야 함.
# 정확도는 높을수록 더 좋은 수치임. -1 * 정확도를 곱해서 큰 정확도 값일수록 최소가 되도록 변환
def objective_func(search_space):
    # 수행 시간 절약을 위해 nestimators는 100으로 축소
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'],
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)

    # accuracy는 cv=3 개수만큼 roc-auc 결과를 리스트로 가짐. 이를 평균해서 반환하되 -1을 곱함.
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}


In [5]:
from hyperopt import fmin, tpe, Trials

trial_val = Trials()
best = fmin(fn = objective_func,
            space = xgb_search_space,
            algo = tpe.suggest,
            max_evals = 40, # 최대 반복 수행횟수
            trials = trial_val, rstate = np.random.default_rng(seed = 9))
print('best:', best)


100%|███████████████████████████████████████████████| 40/40 [00:10<00:00,  3.73trial/s, best loss: -0.9859633569739952]
best: {'colsample_bytree': 0.500542901278722, 'learning_rate': 0.10327937185006039, 'max_depth': 15.0, 'min_child_weight': 1.0}


아래의 코드를 실행하여 출력해주세요

In [6]:
print('colsample_bytree:{0}, learning_rate:{1}, max_depth:{2}, min_child_weight:{3}'.format(
    round(best['colsample_bytree'], 5), round(best['learning_rate'], 5),
    int(best['max_depth']), int(best['min_child_weight'])))

colsample_bytree:0.50054, learning_rate:0.10328, max_depth:15, min_child_weight:1


# 2. 언더 샘플링과 오버 샘플링(20점)

레이블이 불균형한 분포를 가지는 데이터 세트를 학습 시킬 때 예측 성능의 문제가 발생할 수 있습니다. 이때 문에 지도학습에서 극도로 불균형 레이블 값 분포로 인한 문제점을 해결하기 위해서는 적절한 학습 데이터를 확보하는 방안이 필요한데 대표적으로 언더 샘플링과 오버 샘플링 있습니다.

2-1 언더 샘플링과 오버 샘플링의 차이점은 무엇일까요? (10점)

데이터가 불균형한 분포를 가질 때 이를 해소할 수 있는 방법이 언더샘플링과 오버샘플링이다.
 
언더샘플링은 불균형한 데이터셋에서 높은 비율을 차지하는 데이터셋의 데이터수를 줄여서 불균형을 해소하는 방법이고\
오버샘플링은 반대로 낮은 비율의 데이터셋의 데이터수를 늘려서 불균형을 해소하는 방법이다.

언더샘플링은 학습에 사용되는 전체 데이터 수를 줄이기 때문에 성능이 떨어질 수 있다.

2-2
SMOTE 오버 샘플링을 적용하려고 하는 일부 코드를 가져왔다. 아래에서 불필요 없는 코드의 번호를 모두 적으세요. (10점)

In [9]:
!pip install imblearn

In [10]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=0)
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (142, 13) (142,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (171, 13) (171,)
SMOTE 적용 후 레이블 값 분포: 
 1    57
0    57
2    57
Name: target, dtype: int64


2, 6, 7, 8 - train 데이터만 가능

3. 스태킹 앙상블 (55점)

3-1스태킹이 배깅및 부스팅과 공통점과 차이점이 각각 있습니다. 각각 서술해주세요. (10점)

스태킹은 개별 알고리즘을 서로 결합해 예측 결과를 도출한다는 점에서 배깅, 부스팅과 공통점이 있지만\
개별 알고리즘으로 예측한 데이터를 기반으로 다시 예측을 수행한다는 점에 있어서 차이를 가진다.

스태킹 모델을 구현해봅시다. 아래코드를 실행해주세요

In [11]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dataset = load_wine()

wine = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
wine['target']= dataset.target
X_features = wine.iloc[:, :-1]
y_label =  wine.iloc[:, -1]

X_train , X_test , y_train , y_test = train_test_split(X_features , y_label , test_size=0.2 , random_state=0)

3-2 개별 모델로, knn, 랜덤포레스트, 결정트리, 에이다부스트를 사용할것입니다. 모델을 생성한 후, 학습하고, 예측하여 각각의 정확도를 파악해주세요
아래 코드를 사용해주세요 (20점)

knn_clf  = KNeighborsClassifier(n_neighbors=4)

rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

In [12]:
# 개별 ML 모델 생성
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

# 개별 모델들을 학습
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

# 개별 모델 예측 및 정확도 측정
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print('KNN 정확도: {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('RF 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('DT 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('ADA 정확도: {0:.4f}'.format(accuracy_score(y_test, ada_pred)))

KNN 정확도: 0.7500
RF 정확도: 0.9722
DT 정확도: 0.9722
ADA 정확도: 0.8333


3-3 위의 개별 알고리즘으로부터 예측된 값들을 라벨로 옆으로 붙여서 피처 값으로 만들어주세요. 반환된 예측 데이터 세트는 1차원 형태의 ndarry 이므로 numpy의 transpose()를 이용하면 됩니다. (10점)

In [13]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

# transpose를 이용해 전치
pred = np.transpose(pred)
print(pred.shape)

(4, 36)
(36, 4)


3-4 3-2 의 모델의 예측결과를 합한 데이터 세트로 학습/ 예측하는 최종 모델은 로지스틱 회귀 모델입니다. 최종 메타 모델인 로지스틱회귀를 학습하고 예측 정확도를 측정해주세요. (15점)

In [14]:
lr_final = LogisticRegression()
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, final)))

최종 메타 모델의 예측 정확도: 1.0000


-------

## 문제 1. 베이지안 최적화 (25점)

(1) **다음 빈칸 (A),(B)에 들어갈 단어를 적어주세요** - 1점


사이킷런이 제공하는 GridSeachCV API는 분류나 회귀 알고리즘에 사용되는 (A)를 순차적으로 입력하면서 편리하게 최적의 (B)를 도출하는 방안을 제공합니다. 

a : 하이퍼 파라미터
b : 파라미터

(2) 사이킷런에서 제공하는 Grid Search방식이 XGBoost나 LightGBM 알고리즘에서 하이퍼 파라미터 튜닝에 잘 사용되지 않는 이유에 대해서 적어주세요. - 1점

조정할 하이퍼 파라미터 수가 너무 많기 때문에 시간이 오래 걸리기 때문

(3) **다음 빈칸 (C),(D)에 들어갈 단어를 적어주세요** - 1점

베이지안 최적화를 구성하는 두 가지 중요 요소에는 (C)와 (D)가 있다. '

(C)는 (D)로부터 최적 함수를 예측할 수 있는 값을 추천받은 뒤 이를 기반으로 최적 함수 모델을 개선한다.

(D)는 개선된 대체 모델을 기반으로 최적 입력값을 계산한다.

c : 대체 모델
d : 획득 함수

*   베이지안 최적화를 머신러닝 모델의 하이퍼 파라미터 튜닝에 적용할 수 있게 해주는 패키지인 HyperOpt에 대해 알아보자.

아래 코드를 실행해주세요.

In [15]:
# !pip install hyperopt

(4) **다음 빈칸 (A)에 들어갈 단어를 적어주세요** - 1점

HyperOpt는 목적 함수 반환 값의 (A)를 가지는 최적 입력값을 유추한다.

a : 최솟값

### HyperOpt를 이용한 XGBoost 하이퍼 파라미터 최적화

0. 아래 코드를 실행해주세요.

In [16]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score
from hyperopt import hp, STATUS_OK, fmin, tpe, Trials
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

dataset = load_breast_cancer()

cancer_df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
cancer_df['target']= dataset.target
X_features = cancer_df.iloc[:, :-1]
y_label = cancer_df.iloc[:, -1]

1. 데이터 추출을 합니다. - 1점

In [18]:
# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출, random_state=156
X_train, X_test, y_train, y_test= train_test_split(X_features, y_label, test_size=0.2, random_state=156 )

# 앞에서 추출한 학습 데이터를 다시 학습과 검증 데이터로 분리, random_state=156
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=156 )

2. 입력값의 검색공간을 설정해줍니다.(hyperOpt의 함수 사용) - 5점

In [19]:
# max_depth는 4에서 16까지 1간격으로, min_child_weight는 1에서 3까지 1간격으로
# colsample_bytree는 0.4에서 0.8사이, learning_rate는 0.005에서 0.2 사이 정규 분포된 값으로 검색.

xgb_search_space = {'max_depth' : hp.quniform('max_depth', 4, 16, 1),
                    'min_child_weight' : hp.quniform('min_child_weight', 1, 3, 1),
                    'colsample_bytree' : hp.uniform('colsample_bytree', 0.4, 0.8),
                    'learning_rate' : hp.uniform('learning_rate', 0.005, 0.2)}

3. 입력 값에 따른 목적 함수를 설정해줍니다. - 5점

In [20]:
# 검색 공간에서 목적 함수로 입력되는 모든 인자들은 실수형 값이므로 정수형 변환을 해줘야 함
# 정확도는 높을수록 더 좋은 수치인데 최솟값을 가지는 입력값을 유추하니 어떻게 변환하면 좋을까요?
# accuracy는 cross_val_score로 판단(scoring="accuracy"), cv=3

def objective_func(search_space):

    xgb_clf = XGBClassifier(n_estimators=400, max_depth = int(search_space['max_depth']),
                            min_child_weight = int(search_space['min_child_weight']),
                            learning_rate = search_space['learning_rate'],
                            colsample_bytree = search_space['colsample_bytree'],
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}


4. fmin()을 이용해 최적 하이퍼 파라미터를 도출해줍니다. - 5점

In [21]:
# 최대 반복 횟수은 40

trial_val = Trials()
best = fmin(fn=objective_func,
            space=xgb_search_space,
            max_evals=40, 
            algo=tpe.suggest,
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print(best)

100%|███████████████████████████████████████████████| 40/40 [00:22<00:00,  1.80trial/s, best loss: -0.9692546764261647]
{'colsample_bytree': 0.6213906574722501, 'learning_rate': 0.08671934701542167, 'max_depth': 5.0, 'min_child_weight': 2.0}


5. 도출된 최적 하이퍼 파라미터들을 이용해 XGBClassifier를 재학습한 후 성능을 평가해봅니다. - 5점

In [29]:
#learnin_rate와 colsample_bytree는 소수 다섯째자리까지 사용
xgb_wrapper = XGBClassifier(n_estimators=400,
                            learning_rate = 0.08671,
                            max_depth = 5,
                            min_child_weight = 2,
                            colsample_bytree = 0.62139
                           )

evals = [(X_tr, y_tr), (X_val, y_val)]
xgb_wrapper.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='logloss',
                eval_set=evals, verbose=True)

preds = xgb_wrapper.predict(X_test)
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

# accuracy score과 roc auc score을 소수 넷째자리까지 출력
print('accuracy score: {0:.4f}'.format(accuracy_score(preds, y_test)))
print('roc_auc_score: {0:.4f}'.format(roc_auc_score(preds, y_test)))

[0]	validation_0-logloss:0.62034	validation_1-logloss:0.64186
[1]	validation_0-logloss:0.55891	validation_1-logloss:0.59504
[2]	validation_0-logloss:0.50503	validation_1-logloss:0.55529
[3]	validation_0-logloss:0.45905	validation_1-logloss:0.52119
[4]	validation_0-logloss:0.41931	validation_1-logloss:0.49871
[5]	validation_0-logloss:0.38439	validation_1-logloss:0.47909
[6]	validation_0-logloss:0.35320	validation_1-logloss:0.45256
[7]	validation_0-logloss:0.32550	validation_1-logloss:0.43076
[8]	validation_0-logloss:0.30033	validation_1-logloss:0.41120
[9]	validation_0-logloss:0.27697	validation_1-logloss:0.39340
[10]	validation_0-logloss:0.25611	validation_1-logloss:0.37844
[11]	validation_0-logloss:0.23801	validation_1-logloss:0.36968
[12]	validation_0-logloss:0.22207	validation_1-logloss:0.35956
[13]	validation_0-logloss:0.20753	validation_1-logloss:0.34794
[14]	validation_0-logloss:0.19357	validation_1-logloss:0.33872
[15]	validation_0-logloss:0.18123	validation_1-logloss:0.33185
[1

[129]	validation_0-logloss:0.01840	validation_1-logloss:0.25377
[130]	validation_0-logloss:0.01834	validation_1-logloss:0.25369
[131]	validation_0-logloss:0.01829	validation_1-logloss:0.25473
[132]	validation_0-logloss:0.01824	validation_1-logloss:0.25412
[133]	validation_0-logloss:0.01818	validation_1-logloss:0.25445
[134]	validation_0-logloss:0.01813	validation_1-logloss:0.25368
[135]	validation_0-logloss:0.01808	validation_1-logloss:0.25400
[136]	validation_0-logloss:0.01803	validation_1-logloss:0.25262
[137]	validation_0-logloss:0.01798	validation_1-logloss:0.25362
[138]	validation_0-logloss:0.01793	validation_1-logloss:0.25355
[139]	validation_0-logloss:0.01788	validation_1-logloss:0.25411
[140]	validation_0-logloss:0.01783	validation_1-logloss:0.25273
[141]	validation_0-logloss:0.01778	validation_1-logloss:0.25154
[142]	validation_0-logloss:0.01773	validation_1-logloss:0.25253
[143]	validation_0-logloss:0.01768	validation_1-logloss:0.25240
[144]	validation_0-logloss:0.01764	valid

[258]	validation_0-logloss:0.01414	validation_1-logloss:0.24072
[259]	validation_0-logloss:0.01412	validation_1-logloss:0.24087
[260]	validation_0-logloss:0.01410	validation_1-logloss:0.24161
[261]	validation_0-logloss:0.01408	validation_1-logloss:0.24126
[262]	validation_0-logloss:0.01406	validation_1-logloss:0.24114
[263]	validation_0-logloss:0.01405	validation_1-logloss:0.24131
[264]	validation_0-logloss:0.01403	validation_1-logloss:0.24173
[265]	validation_0-logloss:0.01401	validation_1-logloss:0.24146
[266]	validation_0-logloss:0.01399	validation_1-logloss:0.24111
[267]	validation_0-logloss:0.01397	validation_1-logloss:0.24100
[268]	validation_0-logloss:0.01395	validation_1-logloss:0.24141
[269]	validation_0-logloss:0.01393	validation_1-logloss:0.24155
[270]	validation_0-logloss:0.01392	validation_1-logloss:0.24122
[271]	validation_0-logloss:0.01390	validation_1-logloss:0.24136
[272]	validation_0-logloss:0.01388	validation_1-logloss:0.24153
[273]	validation_0-logloss:0.01386	valid

# 문제 2. 스태킹 앙상블(25점)

아래 코드를 실행해주세요.

In [30]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train,X_test,y_train,y_test=train_test_split(X_data,y_label,test_size=0.2,random_state=0)

(1) 스태킹 앙상블에 사용될 KNN, randomforest,decisiontree,adaboost 총 4개의 모델을 생성해주세요. - 2점
- 파라미터: KNN(n_neighbors=4), RF(n_estimators=100, random_state=42), ada(n_estimators=100)을 사용해주세요.

In [31]:
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state = 42)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

(2) 스태킹된 데이터를 로지스틱 회귀를 사용해 최종적으로 학습, 예측할 것입니다. 이를 위한 모델을 생성해주세요. - 1점
- 파라미터: C=10

In [33]:
lr_final = LogisticRegression(C = 10)

아래 코드를 실행시켜주세요.

In [34]:
# 개별 모델들을 학습.
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train , y_train)
dt_clf.fit(X_train , y_train)
ada_clf.fit(X_train, y_train)

# 개별 모델의 예측 데이터 세트 생성
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

(3) 예측 결과를 옆으로 붙여 피처값으로 만들어주세요. (Hint: 예측결과를 먼저 행형태로 붙인 후 행과 열을 바꿔주세요.) - 5점

In [35]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

# transpose를 이용해 전치
pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


(4) 위에서 생성한 데이터 세트를 최종 모델인 로지스틱 회귀로 학습, 예측시킨 후 정확도를 출력해주세요. - 4점

In [36]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, final)))

최종 메타 모델의 예측 정확도: 0.9737


아래는 CV 스태킹 과정입니다.

(5) 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수를 생성해주세요. - 4점


In [45]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits = n_folds, shuffle = False)

    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출
        print('|t 폴드세트: ', folder_counter, ' 시작 ')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
    
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
        model.fit(X_tr, y_tr)
        
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장
        test_pred[:, folder_counter] = model.predict(X_test_n)

    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis = 1).reshape(-1, 1)

    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred, test_pred_mean

(6) 위에서 생성한 함수를 활용해 모델별 학습/테스트 데이터를 분리해주세요. - 2점
(만약 ValueError가 생긴다면, 위 함수에서 KFold의 shuffle 파라미터를 True로 설정해주세요.)

In [46]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier  model 시작 
|t 폴드세트:  0  시작 
|t 폴드세트:  1  시작 
|t 폴드세트:  2  시작 
|t 폴드세트:  3  시작 
|t 폴드세트:  4  시작 
|t 폴드세트:  5  시작 
|t 폴드세트:  6  시작 
RandomForestClassifier  model 시작 
|t 폴드세트:  0  시작 
|t 폴드세트:  1  시작 
|t 폴드세트:  2  시작 
|t 폴드세트:  3  시작 
|t 폴드세트:  4  시작 
|t 폴드세트:  5  시작 
|t 폴드세트:  6  시작 
DecisionTreeClassifier  model 시작 
|t 폴드세트:  0  시작 
|t 폴드세트:  1  시작 
|t 폴드세트:  2  시작 
|t 폴드세트:  3  시작 
|t 폴드세트:  4  시작 
|t 폴드세트:  5  시작 
|t 폴드세트:  6  시작 
AdaBoostClassifier  model 시작 
|t 폴드세트:  0  시작 
|t 폴드세트:  1  시작 
|t 폴드세트:  2  시작 
|t 폴드세트:  3  시작 
|t 폴드세트:  4  시작 
|t 폴드세트:  5  시작 
|t 폴드세트:  6  시작 


(7) np.concatenate를 활용해 train과 test 메타 데이터를 결합해주세요. - 4점

In [47]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis = 1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis = 1)
print('원본 학습 피쳐 데이터 Shape : ', X_train.shape, '원본 테스트 피쳐 Shape : ', X_test.shape)
print('스태킹 학습 피쳐 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피쳐 데이터 Shape:', Stack_final_X_test.shape)

원본 학습 피쳐 데이터 Shape :  (455, 30) 원본 테스트 피쳐 Shape :  (114, 30)
스태킹 학습 피쳐 데이터 Shape: (455, 4) 스태킹 테스트 피쳐 데이터 Shape: (114, 4)


(8) 최종 메타 모델을 학습시킨 뒤, 예측 정확도를 구해주세요. - 3점

In [48]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)
print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, stack_final)))

최종 메타 모델의 예측 정확도: 0.9649
